## 오답노트

In [1]:
import cv2
import mediapipe as mp
from ultralytics import YOLO
import time

# YOLOv8 모델 로드
# 사전에 학습시킨 모델에 ASL 데이터 추가
model = YOLO('asl_yolov8_model.pt') 

# MediaPipe Hands 초기화
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1, # 인식할 수 있는 손은 1개로 초기화
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# 유저로부터 원하는 알파벳 입력받기
target_alphabet = input("원하는 알파벳을 입력하세요: ").upper() # 대문자 처리
print(f"원하는 알파벳: {target_alphabet}")

# 카메라 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

print("카메라가 실행되었습니다. 'q'를 눌러 종료하세요.")

last_wrong_time = 0  # 틀렸을 때의 타임스탬프
wrong_display_duration = 2  # 틀렸을 때 빨간색 상자를 표시할 시간(초)

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 가져올 수 없습니다.")
        break

    # 화면 좌우 반전 (거울 효과)
    frame = cv2.flip(frame, 1)

    # YOLOv8 예측
    results = model.predict(source=frame, conf=0.5, show=False, verbose=False)  # YOLOv8 예측 (로그 숨김)
    detections = results[0].boxes if results[0].boxes else []  # YOLOv8 탐지 결과

    detected_alphabet = None
    for detection in detections:
        coords = detection.xyxy[0].tolist()  # 바운딩 박스 좌표
        x1, y1, x2, y2 = map(int, coords[:4])
        conf = detection.conf[0].item()
        if conf >= 0.6:  # 신뢰도 기준 추가
            class_id = int(detection.cls[0].item())  # 클래스 ID
            detected_alphabet = model.names[class_id].upper()

            # 알파벳 비교 및 처리
            if detected_alphabet == target_alphabet:
                # 동일한 알파벳: 파란색 상자
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                label = f"Correct: {detected_alphabet} {conf:.2f}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                # 확인 메시지 출력
                print(f"정답입니다! 목표 알파벳: {target_alphabet}, 감지된 알파벳: {detected_alphabet}")
            else:
                # 다른 알파벳: 빨간색 상자
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                label = f"Wrong: {detected_alphabet}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                last_wrong_time = time.time()  # 틀린 시간 기록

                # 오답 메시지 출력
                print(f"틀렸습니다! 목표 알파벳: {target_alphabet}, 감지된 알파벳: {detected_alphabet}")

    # MediaPipe 손 랜드마크 감지
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    hand_results = hands.process(rgb_frame)

    # MediaPipe 랜드마크 표시
    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
            )

    # 화면 출력
    cv2.imshow("YOLOv8 + Find_Wrong_Input", frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()
hands.close()


원하는 알파벳: A
카메라가 실행되었습니다. 'q'를 눌러 종료하세요.
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
틀렸습니다! 목표 알파벳: A, 감지된 알파벳: V
